In [173]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

# 1. Load your trained model
model = keras.models.load_model('work/models/gru_model_1.keras')

# 2. Rebuild your char‐level vocabulary/lookups from the training CSV
df = pd.read_csv('work/kaggle_sentiment/tweet_sentiment_train.csv',
                 encoding='utf-8', encoding_errors='replace')
# concatenate all tweets into one long string
text = df['text'].str.cat(sep='\n')

# get sorted list of all unique characters in the corpus
vocab = sorted(set(text))
vocab_size = len(vocab) + 1   # +1 for the OOV token that StringLookup will insert

# make lookup layers exactly as in your notebook
ids_from_chars = keras.layers.StringLookup(
    vocabulary=vocab, mask_token=None  # no PAD token
)
chars_from_ids = keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), 
    invert=True, 
    mask_token=None
)

# helper to turn a string into a vector of IDs
def text_to_ids(s: str):
    # unicode_split → a TF string Tensor of shape (len(s),)
    chars = tf.strings.unicode_split([s], 'UTF-8')
    return ids_from_chars(chars)

# helper to turn a list of IDs back into text
def ids_to_text(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1).numpy().astype(str)

# 3. Seed + pad to the fixed window length your model expects
seed = "Hey, "
seq_length = 100   # <— use whatever you trained with

# convert seed to IDs and pad or trim to length
seed_ids = text_to_ids(seed).numpy()[0]   # shape (len(seed),)
seed_ids = seed_ids[-seq_length:]        # keep last seq_length chars
seed_ids = np.expand_dims(seed_ids, 0)    # make batch of 1
seed_ids = keras.preprocessing.sequence.pad_sequences(
    seed_ids, maxlen=seq_length, padding='pre'
)

# 4. Generate one char at a time
generated_ids = []
num_chars = 512
temperature = 1.0

for _ in range(num_chars):
    # predict next‐char distribution
    preds = model.predict(seed_ids, verbose=0)[0, -1, :]  # (vocab_size,)
    # apply temperature
    preds = np.log(preds + 1e-8) / temperature
    preds = np.exp(preds) / np.sum(np.exp(preds))
    # sample
    next_id = np.random.choice(len(preds), p=preds)
    generated_ids.append(next_id)
    # shift window and append
    seed_ids = np.roll(seed_ids, -1, axis=1)
    seed_ids[0, -1] = next_id

# 5. Decode back to a string
gen_text = ids_to_text(np.array([generated_ids]))
print(seed)

for text in gen_text:
    print(text)


Hey, 
we will work on load poputy to try and wit my BAM awasteph
 I wish I could...but I was nice while trying to make the time next to me against you... I LOVE YOU GUYSS !!  ...I really am headed to shopp. Just Dream You`r Clingy wing wit`s here?
 Dod it`s not the List last year  so thanks  <3
it seemedge you likes your http-can live this year though
if i can`t believe it, only 2 days left for breakfast... sucks
.....  i have much time of year two weeks starting to people around the house  documerton after all t
